# Imports

In [0]:
# built-in utilities
import copy
import os
from glob import glob
import time
from datetime import datetime
from PIL import Image
import sys
import warnings
warnings.filterwarnings("ignore")

# data tools
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, explained_variance_score, mean_squared_log_error, mean_absolute_error, median_absolute_error, mean_squared_error, r2_score, confusion_matrix, roc_curve, accuracy_score, roc_auc_score, homogeneity_score, completeness_score, classification_report, silhouette_samples

# pytorch 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# %load_ext autoreload
# %autoreload 2

!pip install --upgrade grpcio>=1.24.3
!pip install --upgrade google-auth~=1.4.0
!pip install --upgrade tensorflow>=2.0.0
!pip install tensorboard

# colab
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [0]:
# import project tools
import dataLoader

# Sort data

## Binary

In [0]:
# # sort files into folders
# dataLoader.imageSort(
#     destinationRoot="~/Downloads/TrainBinary",
#     imageRoot="~/Downloads/ISIC_2019_Training_Input",
#     labelDf=labelDf,
#     sortType="binary",
# )

In [0]:
image_root = "/content/drive/BinaryData/MEL"
files = glob(os.path.join(image_root, "*.jpg"))
print("Total number of melanoma images: {}\n".format(len(files)))

In [0]:
image_root = "/content/drive/BinaryData/NONMEL"
files = glob(os.path.join(image_root, "*.jpg"))
print("Total number of non-melanoma images: {}\n".format(len(files)))

## Multi class

In [0]:
# # sort files into folders
# dataLoader.imageSort(
#     destinationRoot="~/Downloads/TrainMulti",
#     imageRoot="~/Downloads/ISIC_2019_Training_Input",
#     labelDf=labelDf,
#     sortType="multi",
# )

# Load data

In [0]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor()
])

data_path = "/content/drive/BinaryData"
data = torchvision.datasets.ImageFolder(root=data_path, transform=transform)

print("Classes: {}".format(data.classes))
print("Value counts: {}".format(np.unique(np.array(data.targets), return_counts=True)[1]))

## Visualize raw images

In [0]:
def image_sample(inp, title=None, figsize=(20,20)):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=figsize)
    plt.imshow(inp, interpolation='nearest')
    if title is not None:
        plt.title(title)

# create data loader to visualize image batch grid
loader = torch.utils.data.DataLoader(
    data,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

inputs, classes = next(iter(loader))
out = torchvision.utils.make_grid(inputs)

image_sample(out)

## Class labels

- MEL: “Melanoma” diagnosis confidence
- NV: “Melanocytic nevus” diagnosis confidence
- BCC: “Basal cell carcinoma” diagnosis confidence
- AK: “Actinic keratosis” diagnosis confidence
- BKL: “Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)” diagnosis confidence
- DF: “Dermatofibroma” diagnosis confidence
- VASC: “Vascular lesion” diagnosis confidence
- SCC: "Squamous cell carcinoma"
- UNK: None of the others / "out of distribution" diagnosis confidence

In [0]:
# load and prepare label dataframe
label_df = dataLoader.processLabelDf(
    file="ISIC_2019_Training_GroundTruth.csv"
)
label_df[:10]

## Train/validation split

### Stratified indexes

In [0]:
# statify by target label
train_ix, validation_ix = train_test_split(
    np.arange(len(data.targets)),
    test_size=0.2,
    random_state=0,
    shuffle=True,
    stratify=data.targets
)

print(np.unique(np.array(data.targets)[train_ix], return_counts=True))
print(np.unique(np.array(data.targets)[validation_ix], return_counts=True))

### Split original data into subsets

In [0]:
## use indexes to subset full dataset
# train
train_samples = [data.imgs[i][0] for i in train_ix]
train_targets = [data.targets[i] for i in train_ix]
train_targets = torch.LongTensor(train_targets)

# validation
validation_samples = [data.imgs[i][0] for i in validation_ix]
validation_targets = [data.targets[i] for i in validation_ix]
validation_targets = torch.LongTensor(validation_targets)

In [0]:
class ISICDatasetTrain(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        target = self.targets[index]

        if self.transform:
            image = self.transform(image)

        return image, target

# training transformations
# normMean = [0.763038, 0.54564667, 0.57004464]
# normStd = [0.14092727, 0.15261286, 0.1699712]

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        # transforms.Normalize(norm_mean,
        #                      norm_std)
    ])

train_data = ISICDatasetTrain(
    image_paths=train_samples,
    targets=train_targets,
    # image_paths=train_samples[:250],
    # targets=train_targets[:250],
    transform=train_transform,
)

In [0]:
class ISICDatasetTest(Dataset):
    def __init__(self, image_paths, targets, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        target = self.targets[index]

        if self.transform:
            image = self.transform(image)

        return image, target

# validation transformations
validation_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    # transforms.Normalize(norm_mean,
    #                      norm_std)
])

validation_data = ISICDatasetTest(
    image_paths=validation_samples,
    targets=validation_targets,
    # image_paths=validation_samples[:250],
    # targets=validation_targets[:250],
    transform=validation_transform,
)

### Class weighting


In [0]:
# weights
sample_count = np.array([len(np.where(train_data.targets==t)[0]) for t in np.unique(train_data.targets)])
weight = 1. / sample_count
indv_weights = np.array([weight[int(t)] for t in train_data.targets])
indv_weights = torch.from_numpy(indv_weights)
print("Class weights: {}".format(indv_weights.unique()))

# create sampler object to be used in data loader
weighted_sampler = torch.utils.data.WeightedRandomSampler(indv_weights.type('torch.FloatTensor'), len(indv_weights))

In [0]:
# # Iterate DataLoader and check class balance for each batch
# demo_loader = DataLoader(
#     trainData, 
#     batch_size=16,
#     num_workers=1,
#     sampler=weighted_sampler
# )

# for i, (x, y) in enumerate(demo_loader):
#     print("batch index {}, 0/1: {}/{}".format(
#         i, (y == 0).sum(), (y == 1).sum()))

### Train/validation data loaders

In [0]:
## create data loaders
# train
train_data_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=16,
    shuffle=False,
    sampler=weighted_sampler
)

# validation
validation_data_loader = torch.utils.data.DataLoader(
    validation_data,
    batch_size=128,
    shuffle=False,
)

### Visualize transformed images

In [0]:
# visualize image batch grid
inputs, classes = next(iter(train_data_loader))
out = torchvision.utils.make_grid(inputs)

image_sample(
    out,
    # title=[data.classes[x] for x in classes]
)

In [0]:
# visualize image batch grid
inputs, classes = next(iter(validation_data_loader))
out = torchvision.utils.make_grid(inputs)

image_sample(out)

# References

https://www.kaggle.com/xinruizhuang/skin-lesion-classification-acc-90-pytorch

https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/04-utils/tensorboard

https://pytorch.org/docs/stable/tensorboard.html

https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce

https://towardsdatascience.com/https-medium-com-dinber19-take-a-deeper-look-at-your-pytorch-model-with-the-new-tensorboard-built-in-513969cf6a72

https://github.com/andyhahaha/Uncertainty-Mnist-with-Pytorch

https://discuss.pytorch.org/t/using-nn-dropout2d-at-eval-time-for-modelling-uncertainty/45274

https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/

https://towardsdatascience.com/making-your-neural-network-say-i-dont-know-bayesian-nns-using-pyro-and-pytorch-b1c24e6ab8cd



# Network

## Model

### Custom

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc1 = nn.Linear(32*56*56, 1024)
        self.fc2 = nn.Linear(1024, 2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 32*56*56)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

### Pretrained

#### VGG 16

In [0]:
model = torchvision.models.vgg16(pretrained=True)

In [0]:
print(model)

In [0]:
print(model.classifier[6])

In [0]:
# freeze model weights
for param in model.parameters():
    param.requires_grad = False

# customize last step of model to the this task
model.classifier[6] = nn.Sequential(
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 2),
)

In [0]:
print(model.classifier)

In [0]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print("{:,} total parameters".format(total_params))
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print("{:,} trainable parameters".format(total_trainable_params))

#### resnet50

In [0]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

## Training loop / validation set evaluation process

In [0]:
class PyTorchTrainer:

    def __init__(self, config):

        # random seed settings
        self.seed = config.seed
        torch.manual_seed(self.seed)
        self.verbose = config.verbose

        # data loaders
        self.train_data_loader = config.train_data_loader
        self.validation_data_loader = config.validation_data_loader

        ## model object creation and device assignment
        self.device = config.device

        # if passing in the name of model Class object
        if isinstance(config.model, type):
            self.model = config.model().to(self.device)
        # if model is already instantiated, or if transfer learning model is used
        else:
            self.model = config.model.to(self.device)

        # name to use when saving model state
        if config.model_name is not None:
            self.model_name = config.model_name
        else:
            self.model_name = "untitled"

        # model training settings
        self.lr = config.lr
        self.epochs = config.epochs
        self.optimizer = config.optimizer(self.model.parameters(), lr=self.lr)
        self.criterion = config.criterion

        self.n_epochs_stop = 5
        self.min_val_loss = np.inf
        self.epochs_no_improve = 0

        ## load previous state
        # use checkpoint to load model state and associated objects
        if config.model_object_dir is not None:
            print(">>> Resuming training...")
            self.model_object_dir = config.model_object_dir

            # establish directory
            self.model_dir = os.path.join(self.model_object_dir, "models")
            self.object_dir = os.path.join(self.model_object_dir, "objects")
            self.log_dir = os.path.join(self.model_object_dir, "logs")
            self.log_train_dir = os.path.join(self.model_object_dir, "logs","train")
            self.log_validation_dir = os.path.join(self.model_object_dir, "logs","validation")
            
            # load model
            self.model.load_state_dict(torch.load(os.path.join(self.model_dir, os.listdir(self.model_dir)[0])))
            self.model = self.model.to(self.device)
            self.model_name = os.listdir(self.model_dir)[0].split(".")[0]
            
            # load statistics objects
            self.running_avg_train_f1 = torch.load(os.path.join(self.object_dir, "running_avg_train_f1.pt"))
            self.running_avg_train_precision = torch.load(os.path.join(self.object_dir, "running_avg_train_precision.pt"))
            self.running_avg_train_recall = torch.load(os.path.join(self.object_dir, "running_avg_train_recall.pt"))
            self.running_avg_train_accuracy = torch.load(os.path.join(self.object_dir, "running_avg_train_accuracy.pt"))
            self.running_avg_train_loss = torch.load(os.path.join(self.object_dir, "running_avg_train_loss.pt"))
            
            self.running_avg_validation_f1 = torch.load(os.path.join(self.object_dir, "running_avg_validation_f1.pt"))
            self.running_avg_validation_precision = torch.load(os.path.join(self.object_dir, "running_avg_validation_precision.pt"))
            self.running_avg_validation_recall = torch.load(os.path.join(self.object_dir, "running_avg_validation_recall.pt"))
            self.running_avg_validation_accuracy = torch.load(os.path.join(self.object_dir, "running_avg_validation_accuracy.pt"))
            self.running_avg_validation_loss = torch.load(os.path.join(self.object_dir, "running_avg_validation_loss.pt"))

            self.globaliter = torch.load(os.path.join(self.object_dir, "globaliter.pt"))
    
        else:
            # directory tree for storing model attributes
            current = datetime.today().strftime('%Y%m%d_%H%M') + "_" + self.model_name
            
            self.model_object_dir = os.path.join(os.getcwd(), "model_objects", current)
            self.model_dir = os.path.join(self.model_object_dir, "models")
            self.object_dir = os.path.join(self.model_object_dir, "objects")
            self.log_dir = os.path.join(self.model_object_dir, "logs")
            self.log_train_dir = os.path.join(self.model_object_dir, "logs","train")
            self.log_validation_dir = os.path.join(self.model_object_dir, "logs","validation")
            
            os.makedirs(self.model_object_dir, exist_ok=True)
            os.makedirs(self.model_dir, exist_ok=True)
            os.makedirs(self.object_dir, exist_ok=True)
            os.makedirs(self.log_dir, exist_ok=True)
            os.makedirs(self.log_train_dir, exist_ok=True)
            os.makedirs(self.log_validation_dir, exist_ok=True)
            
            self.globaliter = 0

        # tensorboard
        self.tensorboard_files = config.tensorboard_files
        if self.tensorboard_files:
            self.train_summary_writer = SummaryWriter(self.log_train_dir)
            self.validation_summary_writer = SummaryWriter(self.log_validation_dir)
        else:
            self.train_summary_writer = None
            self.validation_summary_writer = None
            
        self.beginning_time = time.time()

    def train(self, epoch):
        epoch_preds = []
        epoch_targets = []

        epoch_f1 = []
        epoch_precision = []
        epoch_recall = []
        epoch_accuracy = []
        epoch_loss = []

        self.globaliter += 1
        epoch_beginning_time = time.time()
        
        # sample batch number for data capture
        num_batches = np.floor(len(self.train_data_loader.dataset.image_paths) / self.train_data_loader.batch_size)
        sample_batch_idx = np.random.randint(0, num_batches)

        self.model.train()
        print("*" * 100)
        for batch_idx, (data, target) in enumerate(self.train_data_loader):
            batch_beginning_time = time.time()

            data = data.to(self.device)
            target = target.to(self.device)

            output = self.model(data)
            train_loss = self.criterion(output, target)
            epoch_loss.append(train_loss.item())

            self.optimizer.zero_grad()
            train_loss.backward()
            self.optimizer.step()

            #Metrics
            _, pred = torch.max(output, dim=1)
            epoch_preds = epoch_preds + pred.detach().cpu().numpy().tolist()
            epoch_targets = epoch_targets + target.detach().cpu().numpy().tolist()

            metric_f1 = f1_score(epoch_targets, epoch_preds)
            epoch_f1.append(metric_f1)

            metric_precision = precision_score(epoch_targets, epoch_preds)
            epoch_precision.append(metric_precision)

            metric_recall = recall_score(epoch_targets, epoch_preds)
            epoch_recall.append(metric_recall)

            metric_accuracy = accuracy_score(epoch_targets, epoch_preds)
            epoch_accuracy.append(metric_accuracy)

            # print progress report
            if self.verbose:
                if batch_idx % 50 == 0 and batch_idx > 0:
                    print("\nTrain epoch: {} | Batch: {} | [Processed {}/{} ({:.0f}%)]\n\tLoss: {:.6f} | F1: {:.6f} | Precision: {:.6f} | Recall: {:.6f} | Accuracy: {:.6f}".format(
                        epoch, batch_idx, len(epoch_preds), len(self.train_data_loader.dataset),
                        100. * len(epoch_preds) / len(self.train_data_loader.dataset), train_loss.item(), metric_f1,
                        metric_precision, metric_recall, metric_accuracy))
                    print("\tBatch time elapsed: {}\n".format(self.train_timer(batch_beginning_time, time.time())))
                    print("\n" + "*" * 10)

            # # image batch sample
            # if batch_idx == sample_batch_idx:
            #     image_grid = torchvision.utils.make_grid(data.cpu())
            #     self.train_summary_writer.add_image('train/Sample batch', image_grid, global_step=self.globaliter)
            
        # mark epoch end timestamp
        epoch_ending_time = time.time()

        try:
            self.running_avg_train_f1.append((sum(epoch_f1) / len(epoch_f1)))
            self.running_avg_train_precision.append((sum(epoch_precision) / len(epoch_precision)))
            self.running_avg_train_recall.append((sum(epoch_recall) / len(epoch_recall)))
            self.running_avg_train_accuracy.append((sum(epoch_accuracy) / len(epoch_accuracy)))
            self.running_avg_train_loss.append((sum(epoch_loss) / len(epoch_loss)))

            torch.save(self.running_avg_train_f1, os.path.join(self.object_dir, "running_avg_train_f1.pt"))
            torch.save(self.running_avg_train_precision, os.path.join(self.object_dir, "running_avg_train_precision.pt"))
            torch.save(self.running_avg_train_recall, os.path.join(self.object_dir, "running_avg_train_recall.pt"))
            torch.save(self.running_avg_train_accuracy, os.path.join(self.object_dir, "running_avg_train_accuracy.pt"))
            torch.save(self.running_avg_train_loss, os.path.join(self.object_dir, "running_avg_train_loss.pt"))

            # tensorboard
            if self.tensorboard_files:
                self.train_summary_writer.add_scalar('train/F1', self.running_avg_train_f1[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('train/Precision', self.running_avg_train_precision[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('train/Recall', self.running_avg_train_recall[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('train/Accuracy', self.running_avg_train_accuracy[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('train/Loss', self.running_avg_train_loss[-1], global_step=self.globaliter)

                self.train_summary_writer.add_scalar('F1', self.running_avg_train_f1[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('Precision', self.running_avg_train_precision[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('Recall', self.running_avg_train_recall[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('Accuracy', self.running_avg_train_accuracy[-1], global_step=self.globaliter)
                self.train_summary_writer.add_scalar('Loss', self.running_avg_train_loss[-1], global_step=self.globaliter)
                                
                self.train_summary_writer.flush()

        except AttributeError:
            self.running_avg_train_f1 = [(sum(epoch_f1) / len(epoch_f1))]
            self.running_avg_train_precision = [(sum(epoch_precision) / len(epoch_precision))]
            self.running_avg_train_recall = [(sum(epoch_recall) / len(epoch_recall))]
            self.running_avg_train_accuracy = [(sum(epoch_accuracy) / len(epoch_accuracy))]
            self.running_avg_train_loss = [(sum(epoch_loss) / len(epoch_loss))]

        # print progress report
        if self.verbose:
            print("*" * 10 + "\n")
            print("Train epoch: {} \n\tLoss: {:.6f} | F1: {:.6f} | Precision: {:.6f} | Recall: {:.6f} | Accuracy: {:.6f}\n".format(
                        epoch, self.running_avg_train_loss[-1], self.running_avg_train_f1[-1],
                        self.running_avg_train_precision[-1], self.running_avg_train_recall[-1], self.running_avg_train_accuracy[-1]))
            print("\tEpoch time elapsed: {}".format(self.train_timer(epoch_beginning_time, epoch_ending_time)))
            print("\tTotal time elapsed: {}".format(self.train_timer(self.beginning_time, time.time())))
        
        # capture globaliter
        torch.save(self.globaliter, os.path.join(self.object_dir, "globaliter.pt"))

    def validation(self, epoch):
        epoch_preds = []
        epoch_targets = []

        epoch_f1 = []
        epoch_precision = []
        epoch_recall = []
        epoch_accuracy = []
        epoch_loss = []
        
        # sample batch number for data capture
        num_batches = np.floor(len(self.validation_data_loader.dataset.image_paths) / self.validation_data_loader.batch_size)
        sample_batch_idx = np.random.randint(0, num_batches)

        # turn off gradients
        self.model.eval()
        with torch.no_grad():

            for batch_idx, (data, target) in enumerate(self.validation_data_loader):
                # reshape data as needed and send data to GPU if available
                data = data.to(self.device)
                target = target.to(self.device)

                # generate predictions
                output = self.model(data)

                validation_loss = self.criterion(output, target)
                epoch_loss.append(validation_loss.item())

                #Metrics
                _, pred = torch.max(output, dim=1)
                epoch_preds = epoch_preds + pred.detach().cpu().numpy().tolist()
                epoch_targets = epoch_targets + target.detach().cpu().numpy().tolist()

                metric_f1 = f1_score(epoch_targets, epoch_preds)
                epoch_f1.append(metric_f1)

                metric_precision = precision_score(epoch_targets, epoch_preds)
                epoch_precision.append(metric_precision)

                metric_recall = recall_score(epoch_targets, epoch_preds)
                epoch_recall.append(metric_recall)

                metric_accuracy = accuracy_score(epoch_targets, epoch_preds)
                epoch_accuracy.append(metric_accuracy)
            
            # #
            # if batch_idx == sample_batch_idx:
            #     image_grid = torchvision.utils.make_grid(data.cpu())
            #     self.validation_summary_writer.add_image('validation/Sample batch', image_grid, global_step=self.globaliter)
                
            # 
            try:
                self.running_avg_validation_f1.append((sum(epoch_f1) / len(epoch_f1)))
                self.running_avg_validation_precision.append((sum(epoch_precision) / len(epoch_precision)))
                self.running_avg_validation_recall.append((sum(epoch_recall) / len(epoch_recall)))
                self.running_avg_validation_accuracy.append((sum(epoch_accuracy) / len(epoch_accuracy)))
                self.running_avg_validation_loss.append((sum(epoch_loss) / len(epoch_loss)))

                torch.save(self.running_avg_validation_f1, os.path.join(self.object_dir, "running_avg_validation_f1.pt"))
                torch.save(self.running_avg_validation_precision, os.path.join(self.object_dir, "running_avg_validation_precision.pt"))
                torch.save(self.running_avg_validation_recall, os.path.join(self.object_dir, "running_avg_validation_recall.pt"))
                torch.save(self.running_avg_validation_accuracy, os.path.join(self.object_dir, "running_avg_validation_accuracy.pt"))
                torch.save(self.running_avg_validation_loss, os.path.join(self.object_dir, "running_avg_validation_loss.pt"))

                # tensorboard
                if self.tensorboard_files:
                    # validation panel - one scalar per metric per plot
                    self.validation_summary_writer.add_scalar('validation/F1', self.running_avg_validation_f1[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('validation/Precision', self.running_avg_validation_precision[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('validation/Recall', self.running_avg_validation_recall[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('validation/Accuracy', self.running_avg_validation_accuracy[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('validation/Loss', self.running_avg_validation_loss[-1], global_step=self.globaliter)

                    # metric-specific plots
                    self.validation_summary_writer.add_scalar('F1', self.running_avg_validation_f1[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('Precision', self.running_avg_validation_precision[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('Recall', self.running_avg_validation_recall[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('Accuracy', self.running_avg_validation_accuracy[-1], global_step=self.globaliter)
                    self.validation_summary_writer.add_scalar('Loss', self.running_avg_validation_loss[-1], global_step=self.globaliter)
                    
                    self.validation_summary_writer.flush()

            # create statistics object and continue
            except AttributeError:
                self.running_avg_validation_f1 = [(sum(epoch_f1) / len(epoch_f1))]
                self.running_avg_validation_precision = [(sum(epoch_precision) / len(epoch_precision))]
                self.running_avg_validation_recall = [(sum(epoch_recall) / len(epoch_recall))]
                self.running_avg_validation_accuracy = [(sum(epoch_accuracy) / len(epoch_accuracy))]
                self.running_avg_validation_loss = [(sum(epoch_loss) / len(epoch_loss))]
                
            # print progress report
            if self.verbose:
                print("\nValidation epoch: {} \n\tLoss: {:.6f} | F1: {:.6f} | Precision: {:.6f} | Recall: {:.6f} | Accuracy: {:.6f}\n".format(
                        epoch, self.running_avg_validation_loss[-1], self.running_avg_validation_f1[-1],
                        self.running_avg_validation_precision[-1], self.running_avg_validation_recall[-1], self.running_avg_validation_accuracy[-1]))
        
            # early stopping
            if self.running_avg_validation_loss[-1] < self.min_val_loss:
                # Save the model checkpoint
                torch.save(self.model.state_dict(), os.path.join(self.model_dir, "{}.pt".format(self.model_name)))
                self.epochs_no_improve = 0
                self.min_val_loss = self.running_avg_validation_loss[-1]
                
                if self.verbose:
                    print(">>> Improved - saving model\n\n\n")

            else:
                self.epochs_no_improve += 1
                if self.verbose:
                    print(">>> No improvement - {} consecutive epochs\n\n\n".format(self.epochs_no_improve))
                if self.epochs_no_improve == self.n_epochs_stop:
                    if self.verbose:
                        print("\n!!! Early stopping - {} epochs without improvement\n".format(self.n_epochs_stop))
                self.running_avg_validation_loss = []
                    
    def train_timer(self, start, end):
        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)

## Parameter setup & execute

In [0]:
# set input kwargs as object attributes
class ParamConfig:  
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

# configure all necessary parameters
model_params = ParamConfig(
    model = model,
    model_name = "VGG16",
    model_object_dir = "/content/drive/model_objects/20191202_1622_VGG16",
    # model_object_dir = None,
    optimizer = torch.optim.Adam,
    criterion = F.cross_entropy,
    train_data_loader = train_data_loader,
    validation_data_loader = validation_data_loader,
    cuda = True if torch.cuda.is_available() else False,
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    seed = 0,
    lr = 0.001,
    epochs = 50,
    tensorboard_files = True,
    verbose = True,
)

### fit model
# instantiate model object
trainer = PyTorchTrainer(config=model_params)

# iterate fitting procedure over specified epoch count
for epoch in range(1, trainer.epochs + 1):
    trainer.train(epoch)
    trainer.validation(epoch)
trainer.train_summary_writer.close()
trainer.validation_summary_writer.close()


### Tensorboard

In [0]:
## wrong predictions
# count of incorrect preds
num_incorrect = len(target[pred != target])
ix_incorrect = []
if num_incorrect > 0:
    
    # capture indexes of incorrect predictions
    for ix, (image, t, p) in enumerate(zip(data, target, preds)):
        if t != p:
            ix_incorrect.append(ix)
            img_name = 'train/epoch-{}/batch-{}/prediction-{}/label-{}'.format(
                epoch,
                batch_idx,
                labels_dict[p.cpu().item()],
                labels_dict[t.cpu().item()]
            )
            self.train_summary_writer.add_image(img_name, image, global_step=str(epoch) + str(batch_idx) + str(ix))
    
    #     writer.add_image(img_name, image, epoch)
    #     i += 1


In [0]:
## demo code
model = model.to("cuda")
labels_dict = {0: 'MEL', 1: 'NONMEL'}

break_num = 1
for batch_idx, (data, target) in enumerate(train_data_loader):
    print()
    print(batch_idx)
    
    # reshape data as needed and send data to GPU if available
    data = data.to("cuda")
    target = target.to("cuda")
    preds = model(data)
    preds = torch.argmax(F.softmax(preds), dim=1)

    # count of incorrect preds
    num_incorrect = len(target[preds != target])

    ix_incorrect = []
    if num_incorrect > 0:
        
        # capture indexes of incorrect predictions
        for ix, (image, t, p) in enumerate(zip(data, target, preds)):
            if t != p:
                ix_incorrect.append(ix)
                img_name = 'Validation/Prediction-{}/Label-{}'.format(labels_dict[p.cpu().item()], labels_dict[t.cpu().item()])
                print(img_name)
                image_sample(image.cpu(), figsize = (3,3))
                plt.show()
        #     writer.add_image(img_name, image, epoch)
        #     i += 1
    print(target)
    print(preds)
    print(ix_incorrect)
    
    
    if break_num == batch_idx:
        break

# Evaluate

## Reload model & objects

In [0]:
path = "/content/drive/model_objects/20191202_1622_VGG16"
os.listdir(path)

### VGG16

In [0]:
model = torchvision.models.vgg16(pretrained=True)

# freeze model weights
for param in model.parameters():
    param.requires_grad = False

# customize last step of model to the this task
model.classifier[6] = nn.Sequential(
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 2),
)

## General

In [0]:
model_object_dir = "/content/drive/model_objects/20191202_1622_VGG16"

# establish directory
model_dir = os.path.join(model_object_dir, "models")
object_dir = os.path.join(model_object_dir, "objects")
log_dir = os.path.join(model_object_dir, "logs")
log_train_dir = os.path.join(model_object_dir, "logs", "train")
log_val_dir = os.path.join(model_object_dir, "logs", "validation")

# load model
model.load_state_dict(torch.load(os.path.join(model_dir, os.listdir(model_dir)[0])))
model = model.to("cuda")
model_name = os.listdir(model_dir)[0].split(".")[0]

# load statistics objects
running_avg_train_f1 = torch.load(os.path.join(object_dir, "running_avg_train_f1.pt"))
running_avg_train_precision = torch.load(os.path.join(object_dir, "running_avg_train_precision.pt"))
running_avg_train_recall = torch.load(os.path.join(object_dir, "running_avg_train_recall.pt"))
running_avg_train_accuracy = torch.load(os.path.join(object_dir, "running_avg_train_accuracy.pt"))
running_avg_train_loss = torch.load(os.path.join(object_dir, "running_avg_train_loss.pt"))

running_avg_validation_f1 = torch.load(os.path.join(object_dir, "running_avg_validation_f1.pt"))
running_avg_validation_precision = torch.load(os.path.join(object_dir, "running_avg_validation_precision.pt"))
running_avg_validation_recall = torch.load(os.path.join(object_dir, "running_avg_validation_recall.pt"))
running_avg_validation_accuracy = torch.load(os.path.join(object_dir, "running_avg_validation_accuracy.pt"))
running_avg_validation_loss = torch.load(os.path.join(object_dir, "running_avg_validation_loss.pt"))

globaliter = torch.load(os.path.join(object_dir, "globaliter.pt"))

In [0]:
globaliter

In [0]:
plt.plot(running_avg_train_accuracy)

In [0]:
plt.plot(running_avg_validation_accuracy)

In [0]:
plt.plot(running_avg_validation_loss)

In [0]:
# load model
PATH = "models/mnist_hw1_q1_2.pt"
model = fcNet().to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

In [0]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

# Experiments

## Visualizing Convolution Neural Networks using Pytorch

https://towardsdatascience.com/visualizing-convolution-neural-networks-using-pytorch-3dfa8443e74e

### Filter visualization

In [0]:
import torchvision.models as models
alexnet = models.alexnet(pretrained=True)

In [0]:
def plot_weights(model, layer_num, single_channel = True, collated = False):
    """
    model — Alexnet model or any trained model
    layer_num — Convolution Layer number to visualize the weights
    single_channel — Visualization mode
    collated — Applicable for single-channel visualization only.
    """
    #extracting the model features at the particular layer number
    layer = model.features[layer_num]

    #checking whether the layer is convolution layer or not 
    if isinstance(layer, nn.Conv2d):
    #getting the weight tensor data
    weight_tensor = model.features[layer_num].weight.data

    if single_channel:
        if collated:
        plot_filters_single_channel_big(weight_tensor)
        else:
        plot_filters_single_channel(weight_tensor)
        
    else:
        if weight_tensor.shape[1] == 3:
        plot_filters_multi_channel(weight_tensor)
        else:
        print("Can only plot weights with three channels with single channel = False")
        
    else:
    print("Can only visualize layers which are convolutional")
        
#visualize weights for alexnet - first conv layer
plot_weights(alexnet, 0, single_channel = False)

In [0]:
#getting the weight tensor data
weight_tensor = model.features[layer_num].weight.data

In [0]:
#visualize weights for alexnet — first conv layer
plot_weights(alexnet, 0, single_channel = False)

### Image Occlusion Experiments

In [0]:
#for visualization we will use vgg16 pretrained on imagenet data
model = models.vgg16(pretrained=True)

In [0]:
def occlusion(model, image, label, occ_size = 50, occ_stride = 50, occ_pixel = 0.5):
  
    #get the width and height of the image
    width, height = image.shape[-2], image.shape[-1]
  
    #setting the output image width and height
    output_height = int(np.ceil((height-occ_size)/occ_stride))
    output_width = int(np.ceil((width-occ_size)/occ_stride))
  
    #create a white image of sizes we defined
    heatmap = torch.zeros((output_height, output_width))
    
    #iterate all the pixels in each column
    for h in range(0, height):
        for w in range(0, width):
            
            h_start = h*occ_stride
            w_start = w*occ_stride
            h_end = min(height, h_start + occ_size)
            w_end = min(width, w_start + occ_size)
            
            if (w_end) >= width or (h_end) >= height:
                continue
            
            input_image = image.clone().detach()
            
            #replacing all the pixel information in the image with occ_pixel(grey) in the specified location
            input_image[:, :, w_start:w_end, h_start:h_end] = occ_pixel
            
            #run inference on modified image
            output = model(input_image)
            output = nn.functional.softmax(output, dim=1)
            prob = output.tolist()[0][label]
            
            #setting the heatmap location to probability value
            heatmap[h, w] = prob 

    return heatmap

In [0]:

#compute occlusion heatmap
heatmap = occlusion(model, images, pred[0].item(), 32, 14)

#displaying the image using seaborn heatmap and also setting the maximum value of gradient to probability
imgplot = sns.heatmap(heatmap, xticklabels=False, yticklabels=False, vmax=prob_no_occ)
figure = imgplot.get_figure()

## How to train an image classifier

https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5

In [0]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [0]:
def get_random_images(num):
    data = datasets.ImageFolder(data_dir, transform=test_transforms)
    classes = data.classes
    indices = list(range(len(data)))
    np.random.shuffle(indices)
    idx = indices[:num]
    from torch.utils.data.sampler import SubsetRandomSampler
    sampler = SubsetRandomSampler(idx)
    loader = torch.utils.data.DataLoader(data, 
                   sampler=sampler, batch_size=num)
    dataiter = iter(loader)
    images, labels = dataiter.next()
    return images, labels

In [0]:
to_pil = transforms.ToPILImage()
images, labels = get_random_images(5)
fig=plt.figure(figsize=(10,10))
for ii in range(len(images)):
    image = to_pil(images[ii])
    index = predict_image(image)
    sub = fig.add_subplot(1, len(images), ii+1)
    res = int(labels[ii]) == index
    sub.set_title(str(classes[index]) + ":" + str(res))
    plt.axis('off')
    plt.imshow(image)
plt.show()

## Imbalanced data

## Captum insights

https://towardsdatascience.com/facebook-has-been-quietly-open-sourcing-some-amazing-deep-learning-capabilities-for-pytorch-a7ed5bc71f26


## Extract edge features

https://www.analyticsvidhya.com/blog/2019/08/3-techniques-extract-features-from-image-data-machine-learning-python/

In [0]:
#importing the required libraries
import numpy as np
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
import matplotlib.pyplot as plt
%matplotlib inline

#reading the image 
image = imread('puppy.jpeg',as_gray=True)

#calculating horizontal edges using prewitt kernel
edges_prewitt_horizontal = prewitt_h(image)
#calculating vertical edges using prewitt kernel
edges_prewitt_vertical = prewitt_v(image)

imshow(edges_prewitt_vertical, cmap='gray')

## Interpreting deep learning models

https://medium.com/google-developer-experts/interpreting-deep-learning-models-for-computer-vision-f95683e23c1d

### SHAP gradient explainer

In [0]:
import numpy as np
import shap
import keras.backend as K
import json

shap.initjs()


# utility function to visualize SHAP values in larger image formats
# this modifies the `shap.image_plot(...)` function
def visualize_model_decisions(shap_values, x, labels=None, figsize=(20, 30)):
    
    colors = []
    for l in np.linspace(1, 0, 100):
        colors.append((30./255, 136./255, 229./255,l))
    for l in np.linspace(0, 1, 100):
        colors.append((255./255, 13./255, 87./255,l))
    red_transparent_blue = LinearSegmentedColormap.from_list("red_transparent_blue", colors)

    multi_output = True
    if type(shap_values) != list:
        multi_output = False
        shap_values = [shap_values]

    # make sure labels
    if labels is not None:
        assert labels.shape[0] == shap_values[0].shape[0], "Labels must have same row count as shap_values arrays!"
        if multi_output:
            assert labels.shape[1] == len(shap_values), "Labels must have a column for each output in shap_values!"
        else:
            assert len(labels.shape) == 1, "Labels must be a vector for single output shap_values."

    # plot our explanations
    fig_size = figsize
    fig, axes = plt.subplots(nrows=x.shape[0], ncols=len(shap_values) + 1, figsize=fig_size)
    if len(axes.shape) == 1:
        axes = axes.reshape(1,axes.size)
    for row in range(x.shape[0]):
        x_curr = x[row].copy()

        # make sure
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 1:
            x_curr = x_curr.reshape(x_curr.shape[:2])
        if x_curr.max() > 1:
            x_curr /= 255.
        
        axes[row,0].imshow(x_curr)
        axes[row,0].axis('off')
        
        # get a grayscale version of the image
        if len(x_curr.shape) == 3 and x_curr.shape[2] == 3:
            x_curr_gray = (0.2989 * x_curr[:,:,0] + 0.5870 * x_curr[:,:,1] + 0.1140 * x_curr[:,:,2]) # rgb to gray
        else:
            x_curr_gray = x_curr

        if len(shap_values[0][row].shape) == 2:
            abs_vals = np.stack([np.abs(shap_values[i]) for i in range(len(shap_values))], 0).flatten()
        else:
            abs_vals = np.stack([np.abs(shap_values[i].sum(-1)) for i in range(len(shap_values))], 0).flatten()
        max_val = np.nanpercentile(abs_vals, 99.9)
        for i in range(len(shap_values)):
            if labels is not None:
                axes[row,i+1].set_title(labels[row,i])
            sv = shap_values[i][row] if len(shap_values[i][row].shape) == 2 else shap_values[i][row].sum(-1)
            axes[row,i+1].imshow(x_curr_gray, cmap=plt.get_cmap('gray'), alpha=0.15, extent=(-1, sv.shape[0], sv.shape[1], -1))
            im = axes[row,i+1].imshow(sv, cmap=red_transparent_blue, vmin=-max_val, vmax=max_val)
            axes[row,i+1].axis('off')
        
    cb = fig.colorbar(im, ax=np.ravel(axes).tolist(), label="SHAP value", orientation="horizontal", aspect=fig_size[0]/0.2)
    cb.outline.set_visible(False)

In [0]:
model = VGG16(weights='imagenet', include_top=True)
model.summary()

In [0]:
# get imagenet id to label name mappings
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)
    
# make model predictions
predictions = model.predict(preprocess_input(to_predict.copy()))

# get prediction labels
predicted_labels = [class_names.get(str(pred)) for pred in np.argmax(predictions, axis=1)]
print(predicted_labels)

In [0]:
# utility function to pass inputs to specific model layers
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)
    
# focus on the 7th layer of CNN model
print(model.layers[7].input)
Out [46]: <tf.Tensor 'block2_pool_2/MaxPool:0' shape=(?, 56, 56, 128) dtype=float32>

# make model predictions
e = shap.GradientExplainer((model.layers[7].input, model.layers[-1].output), 
                            map2layer(preprocess_input(X.copy()), 7))
shap_values, indexes = e.shap_values(map2layer(to_predict, 7), ranked_outputs=2)
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)
print(index_names)
Out [47]: array([['chain', 'chain_mail'],
                 ['great_grey_owl', 'prairie_chicken'],
                 ['desktop_computer', 'screen'],
                 ['Egyptian_cat', 'tabby']], dtype='<U16')

# visualize model decisions
visualize_model_decisions(shap_values=shap_values, x=to_predict, 
                          labels=index_names, figsize=(20, 40))

In [0]:
# focus on 14th layer of the CNN model
print(model.layers[14].input)
Out [49]: <tf.Tensor 'block4_conv3_2/Relu:0' shape=(?, 28, 28, 512) dtype=float32>

# make model predictions
e = shap.GradientExplainer((model.layers[14].input, model.layers[-1].output), 
                            map2layer(preprocess_input(X.copy()), 14))
shap_values, indexes = e.shap_values(map2layer(to_predict, 14), ranked_outputs=2)
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# visualize model decisions
visualize_model_decisions(shap_values=shap_values, x=to_predict, 
                          labels=index_names, figsize=(20, 40))